# Model Training and Evaluation

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import joblib
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('../data/cleaned/final.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges,age_group,bmi_category
0,19,female,27.900,0,yes,southwest,16884.92400,19-36,Overweight
1,18,male,33.770,1,no,southeast,1725.55230,0-18,Obese I
2,28,male,33.000,3,no,southeast,4449.46200,19-36,Obese I
3,33,male,22.705,0,no,northwest,21984.47061,19-36,Normal
4,32,male,28.880,0,no,northwest,3866.85520,19-36,Overweight


#### Dropping the columns that were created for EDA

In [4]:
df = df.drop(columns=['age_group','bmi_category'])

In [5]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


#### Feature Encoding - Dummy Encoding

In [6]:
df_dummies = pd.get_dummies(df, drop_first=True)
df_dummies.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,False,True,False,False,True
1,18,33.770,1,1725.55230,True,False,False,True,False
2,28,33.000,3,4449.46200,True,False,False,True,False
3,33,22.705,0,21984.47061,True,False,True,False,False
4,32,28.880,0,3866.85520,True,False,True,False,False


#### Creating X and y variables

In [7]:
X = df_dummies.drop('charges', axis=1)
y = df_dummies['charges']

#### Splitting the data in Train and Test Split - (80-20)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

#### Feature Scaling - Standardization

In [9]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

Helper Function to evaluate model

In [10]:
def evaluate(y_true, y_pred, name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name}:")
    print(f"  R²   = {r2:.4f}")
    print(f"  MAE  = {mae:.2f}")
    print(f"  RMSE = {rmse:.2f}\n")

#### **Linear Regression** 

In [11]:
model_lr = LinearRegression()
model_lr.fit(X_train_sc, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [12]:
y_pred_lr = model_lr.predict(X_test_sc)

In [13]:
evaluate( y_test, y_pred_lr,"LinearRegression")

LinearRegression:
  R²   = 0.8069
  MAE  = 4177.05
  RMSE = 5956.34



The Linear Regression model explained 80.7% of the variance in insurance charges, with an average prediction error of $4177 and RMSE of $5956.

#### **RandomForestRegressor**

In [14]:
model_rf = RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_leaf=3, random_state=42)
model_rf.fit(X_train_sc, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,3
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [15]:
y_pred_rf = model_rf.predict(X_test_sc)

In [16]:
evaluate( y_test,y_pred_rf, "RandomForestRegressor")

RandomForestRegressor:
  R²   = 0.9022
  MAE  = 2452.87
  RMSE = 4239.18



The RandomForestRegressor model explained 90.2% of the variance in insurance charges, with an average prediction error of $2453 and RMSE of $4239.

#### **GradientBoostingRegressor**

In [17]:
model_gb = GradientBoostingRegressor(n_estimators=100, max_depth=4, max_leaf_nodes=3)
model_gb.fit(X_train_sc, y_train)

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,4
,min_impurity_decrease,0.0
,init,None


In [18]:
y_pref_gb = model_gb.predict(X_test_sc)

In [19]:
evaluate(y_test, y_pref_gb, "GradientBoostingRegressor")

GradientBoostingRegressor:
  R²   = 0.9007
  MAE  = 2529.47
  RMSE = 4272.40



The GradientBoostingRegressor model explained 90% of the variance in insurance charges, with an average prediction error of $2529 and RMSE of $4572.

### **Hyper Parameter Optimization**

For hyper parameter optimization RandomForestRegressor and GradientBoostingRegressor are selected according to their performance.  
The Hyper parameter Optimization will be done by using RandomizedSearchCV.

Definig the param_grid for RandomForestRegressor

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None,4, 5,6, 18,24],
    'min_samples_split': [2, 3,5],
    'min_samples_leaf': [1, 2, 3,5,4,6],
    'bootstrap': [True, False]
}

In [ ]:
random_search_rf = RandomizedSearchCV(RandomForestRegressor(), param_grid, cv=5, n_iter = 50,random_state=42)
random_search_rf.fit(X_train_sc, y_train)

print(random_search_rf.best_estimator_)

RandomForestRegressor(max_depth=4, min_samples_leaf=3)


In [23]:
model_rf_best = RandomForestRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)
model_rf_best.fit(X_train_sc, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,4
,min_samples_split,3
,min_samples_leaf,4
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [24]:
y_pred_rf_best = model_rf_best.predict(X_test_sc)

In [26]:
evaluate(y_test, y_pred_rf_best, "RandomForestRegressor")

RandomForestRegressor:
  R²   = 0.9027
  MAE  = 2449.91
  RMSE = 4227.70



The RandomForestRegressor model explained 90.27% of the variance in insurance charges, with an average prediction error of $2449 and RMSE of $4227.

Defining param_grid for GradientBoostingRegressor

In [ ]:
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None,4, 5,6, 18,24],
    'min_samples_split': [2, 3,5],
    'min_samples_leaf': [1, 2, 3,4,6],
}

In [ ]:
random_search_gb = RandomizedSearchCV(GradientBoostingRegressor(), param_grid_gb, cv=5, n_iter = 50)
random_search_gb.fit(X_train_sc, y_train)

print(random_search_gb.best_estimator_)

GradientBoostingRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)


In [ ]:
model_gb_best = GradientBoostingRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)
model_gb_best.fit(X_train_sc, y_train)

GradientBoostingRegressor(max_depth=4, min_samples_leaf=4, min_samples_split=3)

In [ ]:
y_pred_gb_best = model_rf_best.predict(X_test_sc)

In [ ]:
evaluate(y_test, y_pred_gb_best, "GradientBoostingRegressor")

GradientBoostingRegressor:
  R²   = 0.9027
  MAE  = 2446.69
  RMSE = 4228.97



The Linear Regression model explained 90.27% of the variance in insurance charges, with an average prediction error of $2446 and RMSE of $4228.

#### Verdict - Both the model have performed well but the RandomForestRegressor gave a slight nice performance so based on the performance RandomForestRegressor is selected.

Creation of Model Pipeline and exporting the pickle file

In [30]:


categorical_cols = ['sex', 'region', 'smoker']
numerical_cols = ['age', 'bmi', 'children']


preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(drop='first'), categorical_cols)
])


pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', model_rf_best),
])


pipeline.fit(df.drop('charges', axis=1), df['charges'])


joblib.dump(pipeline, '../models/insurance_price_pipeline.pkl')


['../models/insurance_price_pipeline.pkl']